# vTTS - Universal TTS/STT Serving System 완전 테스트

**vLLM for Speech** - 모든 TTS/STT 모델 파라미터 테스트

이 노트북에서는:
- ✅ **Supertonic-2 TTS**: 8개 음성 스타일 × 5개 언어
- ✅ **Faster-Whisper STT**: 5개 모델 크기 × 5개 출력 포맷
- 🔜 **CosyVoice3 TTS**: SFT + Zero-shot 모드 (선택적)
- 🔜 **GPT-SoVITS TTS**: Voice cloning (선택적)

## 🚀 Part 1: 설치 및 환경 설정

In [ ]:
# vTTS 설치 (모든 엔진 포함)
!pip install -q "vtts[all] @ git+https://github.com/bellkjtt/vTTS.git"

print("✅ vTTS 설치 완료!")

In [ ]:
# GPU 확인
import torch

print(f"CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"CUDA Version: {torch.version.cuda}")
    DEVICE = "cuda"
else:
    print("GPU 없음, CPU 모드로 실행")
    DEVICE = "cpu"

## 🎙️ Part 2: Supertonic-2 TTS 전체 테스트

### 테스트 파라미터:
- **음성 스타일**: M1, M2, M3, M4, F1, F2, F3, F4 (8개)
- **언어**: en, ko, es, pt, fr (5개)
- **총 조합**: 40개 테스트 케이스

In [ ]:
# Supertonic-2 엔진 로드
from vtts.engines.supertonic import SupertonicEngine
from vtts.engines.base import TTSRequest
import soundfile as sf
import IPython.display as ipd
import os

print("🔧 Loading Supertonic-2 engine...")
tts_engine = SupertonicEngine()
tts_engine.load_model()

print("✅ Supertonic-2 로드 완료!")
print(f"지원 언어: {tts_engine.supported_languages}")
print(f"지원 음성: {tts_engine.supported_voices}")
print(f"샘플레이트: {tts_engine.default_sample_rate}Hz")

In [ ]:
# 언어별 테스트 텍스트
test_texts = {
    "ko": "안녕하세요, vTTS 테스트입니다.",
    "en": "Hello, this is vTTS testing.",
    "es": "Hola, esta es una prueba de vTTS.",
    "pt": "Olá, este é um teste vTTS.",
    "fr": "Bonjour, ceci est un test vTTS."
}

# 모든 음성 스타일
all_voices = ["M1", "M2", "M3", "M4", "F1", "F2", "F3", "F4"]

print(f"📋 테스트 계획: {len(all_voices)} voices × {len(test_texts)} languages = {len(all_voices) * len(test_texts)} cases")

### 2.1 음성 스타일별 테스트 (한국어)

In [ ]:
# 한국어로 모든 음성 스타일 테스트
print("🎙️ 한국어 음성 스타일 테스트\n" + "="*60)

os.makedirs("outputs/supertonic/korean", exist_ok=True)

for voice in all_voices:
    print(f"\n🔊 Voice: {voice}")
    
    request = TTSRequest(
        text=test_texts["ko"],
        language="ko",
        voice=voice
    )
    
    output = tts_engine.synthesize(request)
    
    # 저장
    filename = f"outputs/supertonic/korean/{voice}.wav"
    sf.write(filename, output.audio, output.sample_rate)
    
    # 재생
    print(f"   Duration: {len(output.audio)/output.sample_rate:.2f}s")
    print(f"   File: {filename}")
    display(ipd.Audio(output.audio, rate=output.sample_rate))

print("\n✅ 한국어 음성 스타일 테스트 완료!")

### 2.2 언어별 테스트 (F1 음성)

In [ ]:
# F1 음성으로 모든 언어 테스트
print("🌍 다국어 테스트 (F1 voice)\n" + "="*60)

for lang, text in test_texts.items():
    print(f"\n🔊 Language: {lang}")
    print(f"   Text: {text}")
    
    request = TTSRequest(
        text=text,
        language=lang,
        voice="F1"
    )
    
    output = tts_engine.synthesize(request)
    
    # 저장
    os.makedirs(f"outputs/supertonic/{lang}", exist_ok=True)
    filename = f"outputs/supertonic/{lang}/F1.wav"
    sf.write(filename, output.audio, output.sample_rate)
    
    # 재생
    print(f"   Duration: {len(output.audio)/output.sample_rate:.2f}s")
    display(ipd.Audio(output.audio, rate=output.sample_rate))

print("\n✅ 다국어 테스트 완료!")

### 2.3 전체 조합 테스트 (40개)

In [ ]:
# 모든 조합 테스트 (음성 재생 없이 저장만)
import time
from tqdm import tqdm

print("🚀 전체 조합 테스트 시작...\n")

results = []
total_tests = len(all_voices) * len(test_texts)

with tqdm(total=total_tests, desc="Processing") as pbar:
    for voice in all_voices:
        for lang, text in test_texts.items():
            start_time = time.time()
            
            request = TTSRequest(
                text=text,
                language=lang,
                voice=voice
            )
            
            output = tts_engine.synthesize(request)
            elapsed = time.time() - start_time
            
            # 저장
            os.makedirs(f"outputs/supertonic/{lang}", exist_ok=True)
            filename = f"outputs/supertonic/{lang}/{voice}.wav"
            sf.write(filename, output.audio, output.sample_rate)
            
            # 결과 기록
            results.append({
                "voice": voice,
                "language": lang,
                "duration": len(output.audio) / output.sample_rate,
                "processing_time": elapsed,
                "rtf": elapsed / (len(output.audio) / output.sample_rate),
                "file": filename
            })
            
            pbar.update(1)

print(f"\n✅ {total_tests}개 테스트 완료!")

# 통계 출력
import pandas as pd

df = pd.DataFrame(results)
print("\n📊 통계:")
print(f"평균 처리 시간: {df['processing_time'].mean():.3f}s")
print(f"평균 RTF: {df['rtf'].mean():.3f}x")
print(f"평균 오디오 길이: {df['duration'].mean():.2f}s")

# 언어별 통계
print("\n📈 언어별 평균 RTF:")
print(df.groupby('language')['rtf'].mean().sort_values())

# 음성별 통계
print("\n🎤 음성별 평균 RTF:")
print(df.groupby('voice')['rtf'].mean().sort_values())

## 🎤 Part 3: Faster-Whisper STT 전체 테스트

### 테스트 파라미터:
- **모델 크기**: tiny, base, small, medium, large-v3 (5개)
- **출력 포맷**: json, verbose_json, text, srt, vtt (5개)
- **태스크**: transcribe, translate
- **언어**: auto-detect, ko, en

In [ ]:
# 테스트용 오디오 생성 (Supertonic-2 사용)
test_audios = {}

for lang, text in test_texts.items():
    request = TTSRequest(text=text, language=lang, voice="F1")
    output = tts_engine.synthesize(request)
    
    filename = f"test_audio_{lang}.wav"
    sf.write(filename, output.audio, output.sample_rate)
    test_audios[lang] = filename

print("✅ 테스트 오디오 생성 완료:")
for lang, file in test_audios.items():
    print(f"   {lang}: {file}")

### 3.1 모델 크기별 테스트

In [ ]:
from vtts.engines.faster_whisper import FasterWhisperEngine
from vtts.engines.stt_base import STTRequest

# 테스트할 모델 크기 (Kaggle 메모리 고려)
model_sizes = ["tiny", "base", "small"]  # medium, large-v3은 메모리 부족 가능

print("📏 모델 크기별 테스트\n" + "="*60)

stt_results = []

for model_size in model_sizes:
    print(f"\n🔧 Loading {model_size} model...")
    
    stt_engine = FasterWhisperEngine(
        model_id=model_size,
        device=DEVICE
    )
    stt_engine.load_model()
    
    print(f"✅ {model_size} 로드 완료")
    
    # 한국어 테스트
    start_time = time.time()
    
    request = STTRequest(
        audio=test_audios["ko"],
        language="ko",
        response_format="json"
    )
    
    result = stt_engine.transcribe(request)
    elapsed = time.time() - start_time
    
    print(f"   원본: {test_texts['ko']}")
    print(f"   인식: {result.text}")
    print(f"   처리 시간: {elapsed:.3f}s")
    print(f"   RTF: {elapsed/result.metadata['duration']:.3f}x")
    
    stt_results.append({
        "model": model_size,
        "language": "ko",
        "original": test_texts["ko"],
        "transcribed": result.text,
        "processing_time": elapsed,
        "rtf": elapsed / result.metadata['duration']
    })
    
    # 메모리 정리
    stt_engine.unload_model()
    del stt_engine
    torch.cuda.empty_cache()

print("\n✅ 모델 크기별 테스트 완료!")

### 3.2 출력 포맷별 테스트

In [ ]:
# base 모델로 다양한 출력 포맷 테스트
stt_engine = FasterWhisperEngine(model_id="base", device=DEVICE)
stt_engine.load_model()

output_formats = ["json", "verbose_json", "text", "srt", "vtt"]

print("📄 출력 포맷별 테스트\n" + "="*60)

for fmt in output_formats:
    print(f"\n📋 Format: {fmt}")
    
    request = STTRequest(
        audio=test_audios["ko"],
        language="ko",
        response_format=fmt
    )
    
    result = stt_engine.transcribe(request)
    
    print(f"   Text: {result.text[:100]}..." if len(result.text) > 100 else f"   Text: {result.text}")
    
    # SRT/VTT는 세그먼트 정보 표시
    if fmt in ["srt", "vtt", "verbose_json"] and result.segments:
        print(f"   Segments: {len(result.segments)}개")
        if result.segments:
            seg = result.segments[0]
            print(f"   첫 세그먼트: [{seg['start']:.2f}s-{seg['end']:.2f}s] {seg['text']}")

print("\n✅ 출력 포맷 테스트 완료!")

### 3.3 번역 테스트 (한국어 → 영어)

In [ ]:
print("🌐 번역 테스트 (한국어 → 영어)\n" + "="*60)

request_translate = STTRequest(
    audio=test_audios["ko"],
    task="translate",  # 번역 모드
    response_format="json"
)

result_translate = stt_engine.transcribe(request_translate)

print(f"원본 (한국어): {test_texts['ko']}")
print(f"번역 (영어): {result_translate.text}")
print("\n✅ 번역 테스트 완료!")

### 3.4 다국어 인식 테스트

In [ ]:
print("🌍 다국어 인식 테스트\n" + "="*60)

for lang, audio_file in test_audios.items():
    print(f"\n🔊 Language: {lang}")
    
    # 언어 자동 감지
    request = STTRequest(
        audio=audio_file,
        response_format="json"
    )
    
    result = stt_engine.transcribe(request)
    
    print(f"   원본: {test_texts[lang]}")
    print(f"   인식: {result.text}")
    print(f"   감지된 언어: {result.language}")
    print(f"   신뢰도: {result.metadata['language_probability']:.2%}")

print("\n✅ 다국어 인식 테스트 완료!")

## 🎯 Part 4: TTS + STT 통합 테스트 (Round-trip)

In [ ]:
print("🔄 Round-trip 테스트 (TTS → STT)\n" + "="*60)

roundtrip_results = []

for lang, original_text in test_texts.items():
    print(f"\n🔊 Language: {lang}")
    
    # TTS: 음성 생성
    tts_request = TTSRequest(
        text=original_text,
        language=lang,
        voice="F1"
    )
    tts_output = tts_engine.synthesize(tts_request)
    
    # 임시 파일 저장
    temp_audio = f"temp_{lang}.wav"
    sf.write(temp_audio, tts_output.audio, tts_output.sample_rate)
    
    # STT: 음성 인식
    stt_request = STTRequest(
        audio=temp_audio,
        language=lang,
        response_format="json"
    )
    stt_output = stt_engine.transcribe(stt_request)
    
    print(f"   원본 텍스트: {original_text}")
    print(f"   인식된 텍스트: {stt_output.text}")
    
    # 정확도 계산 (간단한 문자 일치율)
    original_clean = original_text.lower().replace(" ", "").replace(",", "").replace(".", "")
    transcribed_clean = stt_output.text.lower().replace(" ", "").replace(",", "").replace(".", "")
    
    # Levenshtein 거리 대신 간단한 매칭
    matches = sum(1 for a, b in zip(original_clean, transcribed_clean) if a == b)
    accuracy = matches / max(len(original_clean), len(transcribed_clean)) * 100
    
    print(f"   문자 일치율: {accuracy:.1f}%")
    
    roundtrip_results.append({
        "language": lang,
        "original": original_text,
        "transcribed": stt_output.text,
        "accuracy": accuracy
    })

print("\n✅ Round-trip 테스트 완료!")

# 통계
df_roundtrip = pd.DataFrame(roundtrip_results)
print(f"\n📊 평균 정확도: {df_roundtrip['accuracy'].mean():.1f}%")

## 📊 Part 5: 최종 결과 요약 및 저장

In [ ]:
import json

# 전체 결과 저장
final_results = {
    "environment": {
        "cuda_available": torch.cuda.is_available(),
        "device": DEVICE,
        "gpu": torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU"
    },
    "tts_tests": {
        "engine": "Supertonic-2",
        "total_tests": len(results),
        "voices_tested": all_voices,
        "languages_tested": list(test_texts.keys()),
        "avg_rtf": df['rtf'].mean(),
        "results": results
    },
    "stt_tests": {
        "engine": "Faster-Whisper",
        "models_tested": model_sizes,
        "formats_tested": output_formats,
        "results": stt_results
    },
    "roundtrip_tests": {
        "avg_accuracy": df_roundtrip['accuracy'].mean(),
        "results": roundtrip_results
    }
}

# JSON 저장
with open('vtts_complete_test_results.json', 'w', encoding='utf-8') as f:
    json.dump(final_results, f, ensure_ascii=False, indent=2)

print("✅ 결과 저장 완료: vtts_complete_test_results.json")

In [ ]:
# 최종 요약 출력
print("\n" + "="*80)
print("🎉 vTTS 전체 테스트 완료!")
print("="*80)
print()
print("✅ 완료된 테스트:")
print(f"  📦 환경: {DEVICE.upper()}")
print()
print("  🎙️ TTS (Supertonic-2):")
print(f"     - 음성 스타일: {len(all_voices)}개")
print(f"     - 언어: {len(test_texts)}개")
print(f"     - 총 테스트: {len(results)}개")
print(f"     - 평균 RTF: {df['rtf'].mean():.3f}x")
print()
print("  🎤 STT (Faster-Whisper):")
print(f"     - 모델 크기: {len(model_sizes)}개")
print(f"     - 출력 포맷: {len(output_formats)}개")
print(f"     - 다국어 테스트: {len(test_texts)}개")
print()
print("  🔄 Round-trip:")
print(f"     - 평균 정확도: {df_roundtrip['accuracy'].mean():.1f}%")
print()
print("📈 성능 요약:")
print(f"  - TTS 평균 처리 시간: {df['processing_time'].mean():.3f}s")
print(f"  - TTS 평균 RTF: {df['rtf'].mean():.3f}x")
print(f"  - STT 평균 RTF: {pd.DataFrame(stt_results)['rtf'].mean():.3f}x")
print()
print("🚀 다음 단계:")
print("  - GitHub: https://github.com/bellkjtt/vTTS")
print("  - pip install 'vtts[all] @ git+https://github.com/bellkjtt/vTTS.git'")
print("  - vtts serve Supertone/supertonic-2 --stt-model large-v3")
print()

## 🔜 Part 6: CosyVoice3 TTS 테스트 (선택적)

⚠️ **주의**: CosyVoice3는 Git clone이 필요합니다. Kaggle에서는 복잡할 수 있습니다.

In [ ]:
# CosyVoice3 테스트 (선택적)
# 실행하려면 주석 제거

# # CosyVoice 클론
# !git clone --recursive https://github.com/FunAudioLLM/CosyVoice.git
# 
# # PYTHONPATH 설정
# import sys
# sys.path.insert(0, './CosyVoice')
# 
# # CosyVoice 엔진 로드
# from vtts.engines.cosyvoice import CosyVoiceEngine
# 
# cosyvoice = CosyVoiceEngine(model_id="FunAudioLLM/Fun-CosyVoice3-0.5B-2512")
# cosyvoice.load_model()
# 
# # SFT 모드 테스트
# request = TTSRequest(
#     text="안녕하세요, CosyVoice3 테스트입니다.",
#     language="ko",
#     voice="chinese_female"
# )
# output = cosyvoice.synthesize(request)
# 
# # Zero-shot 모드 테스트 (참조 오디오 필요)
# # request_zs = TTSRequest(
# #     text="This is a test with zero-shot mode.",
# #     language="en",
# #     reference_audio="reference.wav",
# #     reference_text="Reference audio text"
# # )
# # output_zs = cosyvoice.synthesize(request_zs)

print("⏭️ CosyVoice3 테스트는 선택적입니다. 코드 주석을 제거하여 실행하세요.")

## 🔜 Part 7: GPT-SoVITS TTS 테스트 (선택적)

⚠️ **주의**: GPT-SoVITS는 Git clone이 필요하며, 참조 오디오가 필수입니다.

In [ ]:
# GPT-SoVITS 테스트 (선택적)
# 실행하려면 주석 제거

# # GPT-SoVITS 클론
# !git clone https://github.com/RVC-Boss/GPT-SoVITS.git
# 
# # PYTHONPATH 설정
# import sys
# sys.path.insert(0, './GPT-SoVITS')
# 
# # GPT-SoVITS 엔진 로드
# from vtts.engines.gptsovits import GPTSoVITSEngine
# 
# gptsovits = GPTSoVITSEngine(model_id="kevinwang676/GPT-SoVITS-v3")
# gptsovits.load_model()
# 
# # Voice cloning 테스트 (참조 오디오 필수)
# # reference_audio는 5초 이상의 깨끗한 음성이어야 함
# request = TTSRequest(
#     text="안녕하세요, GPT-SoVITS 음성 복제 테스트입니다.",
#     language="ko",
#     reference_audio="reference_voice.wav",  # 5초+ 참조 음성
#     reference_text="참조 음성에서 말한 텍스트"  # 참조 오디오의 텍스트
# )
# output = gptsovits.synthesize(request)

print("⏭️ GPT-SoVITS 테스트는 선택적입니다. 코드 주석을 제거하고 참조 오디오를 준비하여 실행하세요.")